# Happiness Report Visualisations 2021
***
Discovering how data visualisations can help to provide happiness.

Analysis completed :
* Plotly scatter
> * Linear trendline
> * Non-Linear trendline
* Plotly scattermatrix : need to re-define analysis
* Radar plots
> * Seaborn : a lot of effort required to create the plots
> * Plotly : had a seamless construction process which allowed for automation of multiple plots

Main impacts from ladder score are using explained variables
1. Log GDP per capita
2. Social support
3. Healthy life expectancy

In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Display multiple outputs from a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Load data

In [ ]:
df = pd.read_csv('../input/world-happiness-report-2021/world-happiness-report-2021.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
# Quick summary of the dataset. Using the .T method to transpose the result
df.describe(include='all').T

## Visualisations with Plotly

In [ ]:
import plotly.express as px

# Review a bubble graph to understand which regions have a higher ladder score bubble
fig = px.scatter(df,
                x="Logged GDP per capita",
                y="Healthy life expectancy",
                size="Ladder score",
                color="Regional indicator",
                hover_name="Country name",
                size_max=10)
fig.show()

By using the ladder score for the buble size, we don't see much difference in the values as the sizes are all very similar. Having the population size of each country could have helped to show a differential.

In [ ]:
# Testing scatter graph matrix by all columns that could potentially impact the Ladder score
fig = px.scatter_matrix(df, dimensions=['Ladder score',
       'Logged GDP per capita', 'Social support', 'Healthy life expectancy',
       'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption'], color="Regional indicator")
fig.show()

In [ ]:
# Adding a linear trend line
fig = px.scatter(df,
                x="Logged GDP per capita",
                y="Healthy life expectancy",
                size="Ladder score",
                color="Regional indicator",
                hover_name="Country name",
                size_max=10,
                trendline="ols")
fig.show()

In [ ]:
# Retreiving model parameters
results = px.get_trendline_results(fig)
results

In [ ]:
# Extract model fit results from the analysis
results.query("`Regional indicator` == 'Middle East and North Africa'").px_fit_results.iloc[0].summary()

In [ ]:
# Adding a non-linear trend line
fig = px.scatter(df,
                x="Logged GDP per capita",
                y="Healthy life expectancy",
                size="Ladder score",
                color="Regional indicator",
                hover_name="Country name",
                size_max=10,
                trendline="lowess")
fig.show()

In [ ]:
# Review a random sample of records
df.sample(5)

## Radar Graph

### 1. Seaborn

In [ ]:
# Import libraries
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Set up data - using Finland as an example
labels = np.array(['Explained by: Log GDP per capita', 'Explained by: Social support','Explained by: Healthy life expectancy',
       'Explained by: Freedom to make life choices',
       'Explained by: Generosity', 'Explained by: Perceptions of corruption'])
stats = df.loc[0,labels].values

In [ ]:
# Create the angles
angles=np.linspace(0, 2*np.pi, len(labels), endpoint=False) # Set the angle
# close the plot
stats=np.concatenate((stats,[stats[0]]))  # Closed
angles=np.concatenate((angles,[angles[0]]))  # Closed

In [ ]:
# Update the plot with values
ax = plt.subplot(111, polar=True)
ax.plot(angles, stats, 'o-', linewidth=2)  # Draw the plot (or the frame on the radar chart)
plt.xticks(angles[:-1], labels, color='grey', size=7)
ax.fill(angles, stats, alpha=0.25)  #Fulfill the area
ax.set_title(str([df.loc[0,"Country name"]])+" Ladder Score = "+ str([df.loc[0,"Ladder score"]]))  # Set the country name as the title
ax.grid(True)
plt.show()

### 2. Plotly

In [ ]:
# Radar from plotly
stats_new = df.loc[0,labels].values
fig = px.line_polar(df, r=stats_new, theta=labels, line_close=True)
fig.update_traces(fill='toself')

In [ ]:
# Multiple radar testing
import plotly.graph_objects as go

fig = go.Figure()

# Stats for three countries Finland[0], Estonia[38] and India[137]
stats1 = df.loc[0,labels].values
stats2 = df.loc[38,labels].values
stats3 = df.loc[137,labels].values

fig.add_trace(go.Scatterpolar(
      r=stats1,
      theta=labels,
      fill='toself',
      name='Finland'
))
fig.add_trace(go.Scatterpolar(
      r=stats2,
      theta=labels,
      fill='toself',
      name='Estonia'
))
fig.add_trace(go.Scatterpolar(
      r=stats3,
      theta=labels,
      fill='toself',
      name='India'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    )),
  showlegend=True
)

In [ ]:
# Methods to return required parameters to create the radar plot. Previous method had too many hardcoded values.
# The methods below aim to automate the data extraction process

# 1. Extract the country stats
def country_stats(df, index, labels):
    stats = df.loc[index, labels].values
    return stats

# 2. Extract the country name
def country_name(df, index, variable='Country name'):
    name = df.loc[index, variable]
    return name

# 3. Update the trace
def add_to_trace(stats, labels, name):
    return fig.add_trace(go.Scatterpolar(
      r=stats,
      theta=labels,
      fill='toself',
      name=name
    ))

In [ ]:
# Testing to understand the values returned for the methods created

# Output the values for the columns at the index value
country_stats(df, 137, labels)
# Output the country name for the index
country_name(df, 137)

In [ ]:
# Testing the new methods for Finland
fig = go.Figure()

# Create the trace values to update
add_to_trace(country_stats(df, 0, labels), labels, country_name(df, 0))

# Update the figure
fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    )),
  showlegend=True
)

In [ ]:
# Review random list of countries
import random
from numpy import random

# Set the seed so that the same list is returned each time
random.seed(10)
x=random.randint(150, size=(5))
x

In [ ]:
# Radar charts of the random list of countries
fig = go.Figure()

# Loop over the numpy array
for i in x:
    add_to_trace(country_stats(df, i, labels), labels, country_name(df, i))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    )),
  showlegend=True
)